In [24]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from time import strftime
import plotly.express as px
import glob
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [25]:
# Missing Data from November 8th, 2020 to January 11st, 2021
files = os.path.join("C:\\Users\\aaliy\\Documents\\Projects\\GitHub\\energyResearch\\Code\\", "bingData*.csv")
files = glob.glob(files)

bingData = pd.concat(map(pd.read_csv, files))
bingData

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE
0,2019-01-01 00:10:07,2019-01-01 01:10:07,MH,E,8.669
1,2019-01-01 00:10:07,2019-01-01 01:10:07,LA,E,6.808
2,2019-01-01 00:10:07,2019-01-01 01:10:07,KH,E,6.475
3,2019-01-01 00:10:07,2019-01-01 01:10:07,JH,E,8.623
4,2019-01-01 00:10:07,2019-01-01 01:10:07,HE,E,5.472
...,...,...,...,...,...
115590,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,E,36.000
115591,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,H,NaN
115592,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,H,2300.000
115593,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,E,NaN


In [26]:
# DATA CLEANING PROCESS
# Isolate to just Type "E" which refers to Electricity
bingData = bingData[bingData['TYPE'] == 'E']


# Add Time Information to bingData Data Frame
bingData['time'] = pd.to_datetime(bingData['START_TIME'])
bingData['Time'] = pd.to_datetime(bingData['time']).dt.time
bingData['Date'] = pd.to_datetime(bingData['time']).dt.date
bingData['Week'] = bingData['time'].dt.strftime("%U")
bingData['Day'] = bingData['time'].dt.strftime("%j")
bingData['Day'] = pd.to_numeric(bingData['Day'], errors='coerce')
bingData['Weekday'] = bingData['time'].dt.strftime("%w")
bingData['Hour'] = pd.to_datetime(bingData['time']).dt.hour

bingData


<ipython-input-26-91b907e8a854>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-91b907e8a854>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-91b907e8a854>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-91b907e

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Date,Week,Day,Weekday,Hour
0,2019-01-01 00:10:07,2019-01-01 01:10:07,MH,E,8.669,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
1,2019-01-01 00:10:07,2019-01-01 01:10:07,LA,E,6.808,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
2,2019-01-01 00:10:07,2019-01-01 01:10:07,KH,E,6.475,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
3,2019-01-01 00:10:07,2019-01-01 01:10:07,JH,E,8.623,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
4,2019-01-01 00:10:07,2019-01-01 01:10:07,HE,E,5.472,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
115587,2021-12-31 22:10:09,2021-12-31 23:10:09,NH,E,6.750,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22
115588,2021-12-31 22:10:09,2021-12-31 23:10:09,PH,E,6.700,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22
115589,2021-12-31 22:10:09,2021-12-31 23:10:09,SA,E,7.220,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22
115590,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,E,36.000,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22


In [27]:
johnsonData = bingData[bingData['BUILDING_ID'] == 'JS']

unionData = bingData[bingData['BUILDING_ID'] == 'UU']

ESData = bingData[bingData['BUILDING_ID'] == 'ES']



#Found Max Demand for Each Day
idx = johnsonData.groupby(['Date'])['VALUE'].transform(max) == johnsonData['VALUE']
johnsonData = johnsonData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
johnsonData

idx = unionData.groupby(['Date'])['VALUE'].transform(max) == unionData['VALUE']
unionData = unionData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
unionData

idx = ESData.groupby(['Date'])['VALUE'].transform(max) == ESData['VALUE']
ESData = ESData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
ESData

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Date,Week,Day,Weekday,Hour
0,2019-01-01 13:10:01,2019-01-01 14:10:01,ES,E,676.698,2019-01-01 13:10:01,13:10:01,2019-01-01,00,1,2,13
1,2019-01-02 15:10:04,2019-01-02 16:10:04,ES,E,656.705,2019-01-02 15:10:04,15:10:04,2019-01-02,00,2,3,15
2,2019-01-03 15:10:06,2019-01-03 16:10:06,ES,E,650.191,2019-01-03 15:10:06,15:10:06,2019-01-03,00,3,4,15
3,2019-01-04 13:10:07,2019-01-04 14:10:07,ES,E,684.318,2019-01-04 13:10:07,13:10:07,2019-01-04,00,4,5,13
4,2019-01-05 16:10:02,2019-01-05 17:10:02,ES,E,656.558,2019-01-05 16:10:02,16:10:02,2019-01-05,00,5,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...
1013,2021-12-27 17:10:07,2021-12-27 18:10:07,ES,E,458.570,2021-12-27 17:10:07,17:10:07,2021-12-27,52,361,1,17
1014,2021-12-28 17:10:01,2021-12-28 18:10:01,ES,E,480.469,2021-12-28 17:10:01,17:10:01,2021-12-28,52,362,2,17
1015,2021-12-29 19:10:09,2021-12-29 20:10:09,ES,E,467.101,2021-12-29 19:10:09,19:10:09,2021-12-29,52,363,3,19
1016,2021-12-30 18:10:06,2021-12-30 19:10:06,ES,E,465.725,2021-12-30 18:10:06,18:10:06,2021-12-30,52,364,4,18


In [28]:

px.line(x=unionData['Date'],y=unionData['VALUE'])

In [29]:
px.line(x=ESData['Date'],y=ESData['VALUE'])


In [30]:
px.line(x=johnsonData['Date'],y=johnsonData['VALUE'])

In [31]:
#How to deal with that one month of missing data? -> Maybe remove 2020 because covid
#do sklearn multivariable regression + calculate error

In [32]:
#Imported weather Data
weather = pd.read_csv('weatherData2019-2021.csv')
weather = pd.DataFrame({'DateTime':weather['DATE'], 'Temp':weather['HourlyDryBulbTemperature'],'Humidity':weather['HourlyRelativeHumidity']}).dropna()
weather

weather['time'] = pd.to_datetime(weather['DateTime'])
weather['Time'] = pd.to_datetime(weather['time']).dt.time
weather['Date'] = pd.to_datetime(weather['time']).dt.date
weather['Week'] = weather['time'].dt.strftime("%U")
weather['Day'] = weather['time'].dt.strftime("%j")
weather['Day'] = pd.to_numeric(weather['Day'], errors='coerce')
weather['Weekday'] = weather['time'].dt.strftime("%w")
weather['Hour'] = pd.to_datetime(weather['time']).dt.hour

#Found Max Temp for Each Day
idx = weather.groupby(['Date'])['Temp'].transform(max) == weather['Temp']
weather = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)

#Found Max Humidity for Each Day
idx = weather.groupby(['Date'])['Humidity'].transform(max) == weather['Humidity']
weather = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
weather

C:\Users\aaliy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning:

Columns (42,43,50,52,74,76,90) have mixed types.Specify dtype option on import or set low_memory=False.



,DateTime,Temp,Humidity,time,Time,Date,Week,Day,Weekday,Hour
0,2019-01-01T06:53:00,48.0,80.0,2019-01-01 06:53:00,06:53:00,2019-01-01,00,1,2,6
1,2019-01-02T23:53:00,33.0,89.0,2019-01-02 23:53:00,23:53:00,2019-01-02,00,2,3,23
2,2019-01-03T02:51:00,34.0,93.0,2019-01-03 02:51:00,02:51:00,2019-01-03,00,3,4,2
3,2019-01-04T14:53:00,40.0,65.0,2019-01-04 14:53:00,14:53:00,2019-01-04,00,4,5,14
4,2019-01-05T13:53:00,42.0,76.0,2019-01-05 13:53:00,13:53:00,2019-01-05,00,5,6,13
...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23
1092,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11
1093,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20
1094,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20


In [44]:
#MANUAL SOLUTION
#Remove November 8th 2020 to January 11th 2021 (Weather Data index 677-741)
weather_data_adjusted = weather[(weather.index < 677) | (weather.index > 741)]
weather_data_adjusted
#Realized there are other data gaps - maybe try linear interpolation later

#DYNAMIC SOLUTION to remove the data from both data frames if missing data
#Merge johnson and weather data
mergedData = pd.merge(weather, unionData, on=['Date'], how='inner')
mergedData

,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,END_TIME,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y
0,2019-01-01T06:53:00,48.0,80.0,2019-01-01 06:53:00,06:53:00,2019-01-01,00,1,2,6,...,2019-01-01 15:10:02,UU,E,298.09,2019-01-01 14:10:02,14:10:02,00,1,2,14
1,2019-01-02T23:53:00,33.0,89.0,2019-01-02 23:53:00,23:53:00,2019-01-02,00,2,3,23,...,2019-01-02 11:10:09,UU,E,441.87,2019-01-02 10:10:09,10:10:09,00,2,3,10
2,2019-01-03T02:51:00,34.0,93.0,2019-01-03 02:51:00,02:51:00,2019-01-03,00,3,4,2,...,2019-01-03 11:10:03,UU,E,455.50,2019-01-03 10:10:03,10:10:03,00,3,4,10
3,2019-01-04T14:53:00,40.0,65.0,2019-01-04 14:53:00,14:53:00,2019-01-04,00,4,5,14,...,2019-01-04 12:10:10,UU,E,428.00,2019-01-04 11:10:10,11:10:10,00,4,5,11
4,2019-01-05T13:53:00,42.0,76.0,2019-01-05 13:53:00,13:53:00,2019-01-05,00,5,6,13,...,2019-01-05 12:10:05,UU,E,209.54,2019-01-05 11:10:05,11:10:05,00,5,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,2021-12-27 11:10:08,UU,E,203.26,2021-12-27 10:10:08,10:10:08,52,361,1,10
1014,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,2021-12-28 10:10:07,UU,E,202.31,2021-12-28 09:10:07,09:10:07,52,362,2,9
1015,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,2021-12-29 11:10:08,UU,E,213.36,2021-12-29 10:10:08,10:10:08,52,363,3,10
1016,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,2021-12-30 11:10:09,UU,E,199.91,2021-12-30 10:10:09,10:10:09,52,364,4,10


In [60]:
x_train = np.asarray([mergedData['Weekday_y']],mergedData['Temp']).T
y_train = np.asarray(mergedData['VALUE'])


model = LinearRegression()
#calculate beta using train
model.fit(x_train,y_train)

LinearRegression()

In [61]:
model.score(x_train,y_train)

1.3407608859483666e-05

In [53]:
model.coef_

array([0.22853712])

In [54]:
model.intercept_

451.09076051231386

In [ ]:
#I need to find some sort of value for occupancy; academic calendar would be good for that

In [70]:
import requests
from bs4 import BeautifulSoup

In [71]:
URL = "https://www.binghamton.edu/academics/academic-calendar.html"
#I can use Wayback machine to get earlier snapshots of the data
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


In [81]:
results = soup.find(id='academic-calendar')
#Go to Spring -> Go to Month -> Get Date

#Spring 2022
#Summer 2022
#Fall 2022

<div class="event-list day-by-day" id="academic-calendar"><div><div>
</div></div><h2>Spring 2022</h2><div><div>
</div>
</div>
<div class="day-info">
<div class="cal-date">
<div class="cal-icon">
<div class="month">Jan</div>
<div class="date">20-23</div>
<div class="day">Thu</div>
</div>
</div>
<div class="info">
<div class="event">
<div class="title"><a class="no-ext" href="https://www.google.com/calendar/event?eid=NHZwdG1tYmNuMmJxcXUxZnA3czRoZXZwaHQgYmluZ2hhbXRvbi5lZHVfajM5Mmc3MDJscXNnc2FmcGZuZHJsaDkwMmdAZw">Move-in Days: Residence halls open</a></div>
<div class="time"><i class="fa fa-clock-o"></i> All Day</div>
</div>
</div>
</div>
<div class="day-info">
<div class="cal-date">
<div class="cal-icon">
<div class="month">Jan</div>
<div class="date">25</div>
<div class="day">Tue</div>
</div>
</div>
<div class="info">
<div class="event">
<div class="title"><a class="no-ext" href="https://www.google.com/calendar/event?eid=N3JndnZoaWtzaGlwNnNlZWwxbmlwZGg3cW4gYmluZ2hhbXRvbi5lZHVfajM5Mmc3MDJ